<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Webscraping OpenTable with Selenium: Guided Lab

_Authors: Joseph Nelson (DC)_

---

> *Note: this lab is intended to be instructor-guided.*


In today's codealong lab, we will build a scraper using urllib and BeautifulSoup. We will remedy some of the pitfalls of automated scraping by using a a "headless" browser called Selenium.

You will be scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings.

### Before you start, please download the appropriate chromedrive for your system [here](https://chromedriver.storage.googleapis.com/index.html?path=2.36/)

### 1. Inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

### 2. Use `urllib` and `BeautifulSoup` to read the contents of the HTML.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = requests.get(url).text

### 3. Print out the HTML (only print a fraction of it). What is in it?

In [3]:
len(html)

531324

In [4]:
html[0:1000]

'           <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Restaurant Reservation Availability</title>    <meta  name="robots" content="noindex" > </meta>      <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon.ico" type="image/x-icon"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-16.png" sizes="16x16"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-32.png" sizes="32x32"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-48.png" sizes="48x48"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-64.png" sizes="64x64"/><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.4/favicon/favicon-128.png" sizes="128x128"/><link rel="apple-touch-icon-precomposed" sizes="152x15

In [5]:
# This is the raw HTML of the page.

### 4. Use Beautiful Soup to convert the raw HTML into a soup object.

In [6]:
# we need to convert this into a soup object
soup = BeautifulSoup(html, 'html.parser')

### 5. Extract the name of each restaurant.

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? 

> *Hint: we need to know where in the **html** the restaurant element is housed.*

**5.A See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.**

In [7]:
# print the restaurant names
print(soup.find_all('span', {'class': 'rest-row-name-text'})[0:20])

[<span class="rest-row-name-text">Demarco Viaduct</span>, <span class="rest-row-name-text">1027 Eichmann</span>, <span class="rest-row-name-text">Eva O'Keefe</span>, <span class="rest-row-name-text">Pattie Square</span>, <span class="rest-row-name-text">Rerum Weber</span>, <span class="rest-row-name-text">Ramp</span>, <span class="rest-row-name-text">Eves</span>, <span class="rest-row-name-text">Wintheiser</span>, <span class="rest-row-name-text">Laudantium</span>, <span class="rest-row-name-text">Funk</span>, <span class="rest-row-name-text">Brook</span>, <span class="rest-row-name-text">Unde Way</span>, <span class="rest-row-name-text">Drive</span>, <span class="rest-row-name-text">Cathys</span>, <span class="rest-row-name-text">Shore</span>, <span class="rest-row-name-text">Cumque Course</span>, <span class="rest-row-name-text">Odio DuBuque</span>, <span class="rest-row-name-text">Qui Vista</span>, <span class="rest-row-name-text">Centers</span>, <span class="rest-row-name-text">Cul

**5.B Create a list of _only_ the restaurant names (no tags).**


In [8]:
r_names = []

# for each element you find, print out the restaurant name
for entry in soup.find_all('span', {'class': 'rest-row-name-text'}):
    r_names.append(entry.text)

In [9]:
r_names[0:20]

['Demarco Viaduct',
 '1027 Eichmann',
 "Eva O'Keefe",
 'Pattie Square',
 'Rerum Weber',
 'Ramp',
 'Eves',
 'Wintheiser',
 'Laudantium',
 'Funk',
 'Brook',
 'Unde Way',
 'Drive',
 'Cathys',
 'Shore',
 'Cumque Course',
 'Odio DuBuque',
 'Qui Vista',
 'Centers',
 'Culpa']

### 6. Repeat this process but for location.

For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [10]:
# first, see if you can identify the location for all elements -- print it out
print(soup.find_all('span', {'class': 'rest-row-meta--location rest-row-meta-text'})[0:5])

[<span class="rest-row-meta--location rest-row-meta-text">South Jaylonberg</span>, <span class="rest-row-meta--location rest-row-meta-text">Elbertville</span>, <span class="rest-row-meta--location rest-row-meta-text">West Jammie</span>, <span class="rest-row-meta--location rest-row-meta-text">Haleyfort</span>, <span class="rest-row-meta--location rest-row-meta-text">New Donnyberg</span>]


In [11]:
r_loc = []
for entry in soup.find_all('span', {'class': 'rest-row-meta--location rest-row-meta-text'}):
    r_loc.append(entry.text)
    
r_loc[0:10]

['South Jaylonberg',
 'Elbertville',
 'West Jammie',
 'Haleyfort',
 'New Donnyberg',
 'West Newellton',
 'East Thurmanside',
 'Port Prudence',
 'Ileneton',
 'Emmaberg']

### 7. Get the price for each restaurant.

The price is number of dollar signs on a scale of one to four for each restaurant. We'll follow the same process.

In [12]:
# print out all prices
print(soup.find_all('div', {'class': 'rest-row-pricing'})[0:5])

[<div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>, <div class="rest-row-pricing"> <i>  $    $      </i>   $    $      </div>, <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>, <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>, <div class="rest-row-pricing"> <i>  $    $    $    </i>   $        </div>]


In [13]:
r_dollars = []

# get EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class': 'rest-row-pricing'}):
    r_dollars.append(entry.find('i').text)
    
r_dollars[0:10]

['  $    $      ',
 '  $    $      ',
 '  $    $    $    ',
 '  $    $    $    ',
 '  $    $    $    ',
 '  $    $    $    $  ',
 '  $    $    $    $  ',
 '  $    $    $    $  ',
 '  $    $      ',
 '  $    $    $    ']

**7.B Convert the dollar sign strings to a count of the number of dollar signs.**

Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [14]:
r_dollar_count = []

for entry in soup.find_all('div', {'class': 'rest-row-pricing'}):
    price = entry.find('i').text
    r_dollar_count.append(price.count('$'))
    
r_dollar_count[0:10]

[2, 2, 3, 3, 3, 4, 4, 4, 2, 3]

### 8. Can you find the number of times a restaurant was booked.

In the next cell, print out a sample of objects that contain the number of times the restaurant was booked.

> *Note: if you can't, why do you think this is happening?*

In [15]:
# print out all objects that contain the number of times the restaurant was booked
print(soup.find_all('span', {'class': 'tadpole'})[0:20])

[]


That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [16]:
# let's first try printing out all 'span' class objects
for entry in soup.find_all('span')[0:30]:
    print(entry)

<span class="menu-list-link-meta">4658</span>
<span class="menu-list-link-meta">877</span>
<span class="menu-list-link-meta">2340</span>
<span class="menu-list-link-meta">1441</span>
<span itemprop="name">Home</span>
<span itemprop="item"><span itemprop="name">United States</span></span>
<span itemprop="name">United States</span>
<span itemprop="name">Washington, D.C. Area</span>
<span class="show-filter-text">Show filters</span>
<span class="hide-filter-text">Hide filters</span>
<span class="filters icon-as-button button--no-hover" data-target="sort-filter-menu" id="filter-button-toggle" title="Sort"><span class="sort-icon"></span> <span class="current-sort-order pref-label underline-hover">Sort</span></span>
<span class="sort-icon"></span>
<span class="current-sort-order pref-label underline-hover">Sort</span>
<span>Best Match</span>
<span>A-Z</span>
<span>Highest Rated</span>
<span class="pref-label">List</span>
<span class="pref-label">Map</span>
<span class="rest-row-name-text">De

In [17]:
# Can't find the booking count in the object. This requires javascript.

## Enter Selenium

---

Selenium is a headless browser. That means it enables us to mock human browsing behavior -- even waiting for JavaScript elements to load.

If you do not already have Selenium installed, you can do so via pip. Simply: `pip install selenium`

In [18]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

### 9. What is going to happen when I run the next cell?

The chromedriver has been provided in the 'chromedriver' folder so no reason to download another.

In [19]:
# create a driver called driver
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")

Pretty crazy, right? Let's close that driver. 

In case you're wondering. this should have opened up a new browswer window.  Check all of your desktop displays if you didn't see it automatically pop up. 

In [20]:
# close it
driver.close()

### 10. Use the driver to visit `www.python.org`

In [21]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

### 11. Visit the OpenTable page using the driver

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. 

In the next cell, prove you can programmatically visit the page.

In [22]:
# visit our OpenTable page
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

### 12. Resolve the javascript issue using the driver and find the bookings.

What we can do in this case is:
1. Request that the page load
2. wait one second
3. grab the source html from the page 

Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

**Once you have the HTML with the javascript rendered, repeat the processes above to find the bookings.**

In [23]:
# import sleep
from time import sleep

In [24]:
# visit our relevant page
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")

# wait one second
sleep(1)

#grab the page source
html = driver.page_source

In [25]:
# BeautifulSoup it!
html = BeautifulSoup(html, 'lxml')

In [26]:
# Now, let's return to our earlier problem: how do we locate bookings on the page?

In [27]:
# print out the number bookings for all restaurants
print(html.find_all('div', {'class':'booking'})[0:10])

[<div class="booking"><span class="tadpole"></span>Booked 3 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 417 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 276 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 76 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 190 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 93 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 66 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 46 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 132 times today</div>, <div class="booking"><span class="tadpole"></span>Booked 48 times today</div>]


In [28]:
r_bookings = []
for booking in html.find_all('div', {'class':'booking'}):
    r_bookings.append(booking.text)
    
r_bookings[0:15]

['Booked 3 times today',
 'Booked 417 times today',
 'Booked 276 times today',
 'Booked 76 times today',
 'Booked 190 times today',
 'Booked 93 times today',
 'Booked 66 times today',
 'Booked 46 times today',
 'Booked 132 times today',
 'Booked 48 times today',
 'Booked 66 times today',
 'Booked 37 times today',
 'Booked 32 times today',
 'Booked 46 times today',
 'Booked 77 times today']

In [29]:
# We've succeeded!

# But we can clean this up a little bit. 
# We're going to use regular expressions (regex) to grab only the 
# digits that are available in each of the text.

# The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [30]:
# import regex
import re

In [31]:
# Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [32]:
r_bookings_num = []

# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search('\d+', booking.text)
    
    if match:
        # append if found
        r_bookings_num.append(int(match.group()))
    else:
        # otherwise 0
        r_bookings_num.append(0)
        
r_bookings_num[0:15]

[3, 417, 276, 76, 190, 93, 66, 46, 132, 48, 66, 37, 32, 46, 77]

### 13. Can we get all of the items we want from the page in a single `find_all`?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [33]:
# print out all entries
entries = html.find_all('div', {'class':'result content-section-list-row cf with-times'})

### 14. Does every single entry have each element we want?

In [34]:
# I did this previously. I know for a fact that not every element has a 
# number of recent bookings. That's probably exactly why OpenTable houses 
# this in JavaScript: they want to continously update the number of bookings 
# with the most relevant number of values.

In [35]:
# what happens when a booking is not available?
# print out some booking entries, using the identification code we wrote above
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'})[0:50]:
    print(entry.find('div', {'class':'booking'}))

<div class="booking"><span class="tadpole"></span>Booked 3 times today</div>
None
<div class="booking"><span class="tadpole"></span>Booked 417 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 276 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 76 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 190 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 93 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 66 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 46 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 132 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 48 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 66 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 37 times today</div>
<div class="booking"><span class="tadpole"></span>Booked

### 15. Use python exceptions to handle cases when bookings aren't found.

When a booking is not found, store `'ZERO'`.

In [36]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
entries = []

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    try:
        entries.append(entry.find('div', {'class':'booking'}).text)
    except:
        entries.append('ZERO')
        
print(entries.count('ZERO'))

2


### 16. Putting it all together in a dataframe.

**Loop through each entry. For each entry:**
1. Grab the relevant information we want (name, location, price, bookings). 
2. Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [37]:
# I'm going to create my empty df first
import pandas as pd

dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [38]:
# loop through each entry
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    # grab the name
    name = entry.find('span', {'class': 'rest-row-name-text'}).text
    # grab the location
    location = entry.find('span', {'class': 'rest-row-meta--location rest-row-meta-text'}).text
    # grab the price
    price = entry.find('div', {'class': 'rest-row-pricing'}).find('i').text.count('$')
    # try to find the number of bookings
    try:
        temp = entry.find('div', {'class':'booking'}).text
        match = re.search('\d+', temp)
        if match:
            bookings = match.group()
    except:
        bookings = 'NA'
    
    # add to df
    dc_eats.loc[len(dc_eats)]=[name, location, price, bookings]

In [39]:
# check out our work
dc_eats.head()

,name,location,price,bookings
0,Ruffino's - Arlington,Arlington,2,3
1,Joe's Place Pizza and Pasta,Arlington,2,NA
2,Founding Farmers - DC,Foggy Bottom,2,417
3,Farmers Fishers Bakers,Georgetown,2,276
4,Ambar - Arlington,Arlington,2,76


### 17. [Bonus] Sending keys over the driver.

We can send keys to the page using the driver. Below is a demonstration of how to search the page using the Selenium driver.

In [40]:
# we can send keys as well
from selenium.webdriver.common.keys import Keys

In [41]:
# open the driver
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")

# visit Python
driver.get("http://www.python.org")

# verify we're in the right place
assert "Python" in driver.title

In [42]:
# find the search position
elem = driver.find_element_by_name("q")

In [43]:
# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

In [44]:
# send those keys
elem.send_keys(Keys.RETURN)

# no results
assert "No results found." not in driver.page_source

In [45]:
# close
driver.close()

In [46]:
# all at once:
driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

## Additional resources

---

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html